In [1]:
import pandas as pd
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
import time
from IPython.display import clear_output
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [3]:
#Preprocess training set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/labels.csv')

labelsTrain = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTrain.drop(columns =[0,2], inplace = True)
labelsTrain.columns = [ "eye_color" , "img_name"]
labelsTrain = labelsTrain.astype({'eye_color': 'int32'})
labelsTrain = labelsTrain.astype({'img_name': 'string'})

#Preprocess testing set
data=pd.read_csv('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/labels.csv')

labelsTest = data["\teye_color\tface_shape\tfile_name"].str.split(pat="\t", n=-1, expand=True)
labelsTest.drop(columns =[0,2], inplace = True)
labelsTest.columns = [ "eye_color" , "img_name"]
labelsTest = labelsTest.astype({'eye_color': 'int32'})
labelsTest = labelsTest.astype({'img_name': 'string'})

In [4]:
#Load training set################################
imagesTrain = np.zeros((10000, 173, 155)) 

for i in range(0,10000):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set/img/' + labelsTrain["img_name"][i])
    image = image[75:420,95:405] # cut 95 from left / right sides , 75 from top, 80 from the bottom
    image = cv.resize(image, dsize=(155 , 173), interpolation=cv.INTER_CUBIC)
    imagesTrain[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
imagesTrain = imagesTrain.reshape(10000,26815)    
imagesTrain = pd.DataFrame(imagesTrain) 


#Load Testing set################################
imagesTest = np.zeros((2500, 173, 155))

for i in range(0,2500):
    image = cv.imread('C:/Users/X99S5/Documents/AMLS_22-23_SN18046828/Datasets/cartoon_set_test/img/' + labelsTest["img_name"][i])
    image = image[75:420,95:405]
    image = cv.resize(image, dsize=(155 , 173), interpolation=cv.INTER_CUBIC)
    imagesTest[i] = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    
    
imagesTest = imagesTest.reshape(2500,26815)   
imagesTest = pd.DataFrame(imagesTest) 